In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2021-08-26 04:02:41--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.16.29
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.16.29|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2021-08-26 04:02:41--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?OgfN-ctAeqAXPJ5I3DJJz25lmhUbYAae-0GnIgB1cvO19zFvBDDDWLDeUUFzE9MTZZApisfs4T2LSN8IPH4HPdisIeioKANdQ0ab3IjQv4Yu8Hh

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-af5rpp2m
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-af5rpp2m
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=9b77a2727be3cd45e90331b24341d8ad8458daf649947582fd6339ec368566da
  Stored in directory: /tmp/pip-ephem-wheel-cache-m8a7oso5/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


## **COMPUTE MIN, MAX, SUM, AVERAGE, STD. DEVIATION**

### **MIN, MAX**

In [ ]:
%%cu
#include <iostream>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <time.h>
#include <chrono>
using namespace std;


__global__ void getMinGPU(int *arr, int N, int mul, int offset) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int i = mul * idx, j = i + offset;
    if(j < N) if(arr[i] > arr[j]) arr[i] = arr[j];
}

void minimumGPU(int *arr, int n, int *ans) {
    int mul = 2, offset = 1, N = n;
    while(n > 1) {
        n = (n + 1) / 2;
        int blocks = (n-1) / 256 + 1;
        int threads = 256;
        getMinGPU<<<blocks, threads>>>(arr, N, mul, offset);
        mul *= 2;
        offset *= 2;
    }
    cudaMemcpy(ans, arr, sizeof(int), cudaMemcpyDeviceToHost);
}

void minimumCPU(int *arr, int n, int *ans) {
    *ans = INT_MAX;
    for(int i = 0; i < n; i++) {
        if(arr[i] < (*ans)) {
            (*ans) = arr[i];
        }
    }
}

__global__ void getMaxGPU(int *arr, int N, int mul, int offset) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int i = mul * idx, j = i + offset;
    if(j < N) if(arr[i] < arr[j]) arr[i] = arr[j];
}

void maximumGPU(int *arr, int n, int *ans) {
    int mul = 2, offset = 1, N = n;
    while(n > 1) {
        n = (n + 1) / 2;
        int blocks = (n-1) / 256 + 1;
        int threads = 256;
        getMaxGPU<<<blocks, threads>>>(arr, N, mul, offset);
        mul *= 2;
        offset *= 2;
    }
    cudaMemcpy(ans, arr, sizeof(int), cudaMemcpyDeviceToHost);
}

void maximumCPU(int *arr, int n, int *ans) {
    *ans = INT_MIN;
    for(int i = 0; i < n; i++) {
        if(arr[i] > (*ans)) {
            (*ans) = arr[i];
        }
    }
}

int main() {
    srand(time(0));
    int N = (1<<20);
    int *h_array, *d_array;
    
    h_array = new int[N];
    cudaMalloc((void**)&d_array, N * sizeof(int));

    for(int i = 0; i < N; i++) h_array[i] = rand();
    cudaMemcpy(d_array, h_array, N * sizeof(int), cudaMemcpyHostToDevice);

    int *ans_CPU, *ans_GPU;
    ans_CPU = new int;
    ans_GPU = new int;

    auto start = chrono::steady_clock::now();
    minimumCPU(h_array, N, ans_CPU);
    auto end = chrono::steady_clock::now();

    cout << "CPU STATISTICS:\n";
    cout << "Minimum value: " << *ans_CPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    start = chrono::steady_clock::now();
    maximumCPU(h_array, N, ans_CPU);
    end = chrono::steady_clock::now();

    cout << "Maximum value: " << *ans_CPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n\n";

    start = chrono::steady_clock::now();
    minimumGPU(d_array, N, ans_GPU);
    end = chrono::steady_clock::now();

    cout << "GPU STATISTICS:\n";
    cout << "Minimum value: " << *ans_GPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    cudaMemcpy(d_array, h_array, N * sizeof(int), cudaMemcpyHostToDevice);

    start = chrono::steady_clock::now();
    maximumGPU(d_array, N, ans_GPU);
    end = chrono::steady_clock::now();

    cout << "Maximum value: " << *ans_GPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    return 0;
}

CPU STATISTICS:
Minimum value: 1341
Time taken: 4946 microseconds
Maximum value: 2147479110
Time taken: 4484 microseconds

GPU STATISTICS:
Minimum value: 1341
Time taken: 199 microseconds
Maximum value: 2147479110
Time taken: 252 microseconds



### **SUM, AVERAGE, STD. DEVIATION**

In [ ]:
%%cu
#include <iostream>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <time.h>
#include <chrono>
#include <math.h>
using namespace std;


__global__ void getSumGPU(int *arr, int N, int mul, int offset) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int i = mul * idx, j = i + offset;
    if(j < N) arr[i] += arr[j];
}

__global__ void getStdDevGPU(int *arr, int N, int mean, int mul, int offset) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int i = mul * idx, j = i + offset;
    if(j < N) arr[i] = (arr[i]-mean)*(arr[i]-mean) + (arr[j]-mean)*(arr[j]-mean);
    else if(i < N) arr[i] = (arr[i]-mean)*(arr[i]-mean);
}

void sumGPU(int *arr, int n, int *ans) {
    int mul = 2, offset = 1, N = n;
    while(n > 1) {
        n = (n + 1) / 2;
        int blocks = (n-1) / 256 + 1;
        int threads = 256;
        getSumGPU<<<blocks, threads>>>(arr, N, mul, offset);
        mul *= 2;
        offset *= 2;
    }
    cudaMemcpy(ans, arr, sizeof(int), cudaMemcpyDeviceToHost);
}

void stdDevGPU(int *arr, int n, int mean, int *ans) {
    int mul = 2, offset = 1, N = n;
    bool first = true;
    while(n > 1) {
        n = (n + 1) / 2;
        int blocks = (n-1) / 256 + 1;
        int threads = 256;
        if(first) {
            getStdDevGPU<<<blocks, threads>>>(arr, N, mean, mul, offset);
            first =  false;
        }
        else getSumGPU<<<blocks, threads>>>(arr, N, mul, offset);
        mul *= 2;
        offset *= 2;
    }
    cudaMemcpy(ans, arr, sizeof(int), cudaMemcpyDeviceToHost);
    (*ans) = sqrt((*ans) / N);
}

void sumCPU(int *arr, int n, int *ans) {
    *ans = 0;
    for(int i = 0; i < n; i++) {
        (*ans) += arr[i];
    }
}

void stdDevCPU(int *arr, int n, int mean, int *ans) {
    *ans = 0;
    for(int i = 0; i < n; i++) (*ans) += (arr[i]-mean) * (arr[i]-mean);
    (*ans) = sqrt((*ans) / n);
}

int main() {
    srand(time(0));
    int N = (1<<19);
    int *h_array, *d_array;
    
    h_array = new int[N];
    cudaMalloc((void**)&d_array, N * sizeof(int));

    int maxVal = 1e3;
    for(int i = 0; i < N; i++) h_array[i] = rand() % maxVal;
    cudaMemcpy(d_array, h_array, N * sizeof(int), cudaMemcpyHostToDevice);

    int *ans_CPU, *ans_GPU, meanCPU, meanGPU;
    ans_CPU = new int;
    ans_GPU = new int;

    auto start = chrono::steady_clock::now();
    sumCPU(h_array, N, ans_CPU);
    meanCPU = (*ans_CPU) / N;
    auto end = chrono::steady_clock::now();

    cout << "CPU STATISTICS:\n";
    cout << "Sum value: " << *ans_CPU << '\n';
    cout << "Average value: " << meanCPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    start = chrono::steady_clock::now();
    stdDevCPU(h_array, N, meanCPU, ans_CPU);
    end = chrono::steady_clock::now();

    cout << "Standard deviation value: " << *ans_CPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n\n";

    start = chrono::steady_clock::now();
    sumGPU(d_array, N, ans_GPU);
    meanGPU = (*ans_GPU) / N;
    end = chrono::steady_clock::now();

    cout << "GPU STATISTICS:\n";
    cout << "Sum value: " << *ans_GPU << '\n';
    cout << "Average value: " << meanGPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    cudaMemcpy(d_array, h_array, N * sizeof(int), cudaMemcpyHostToDevice);

    start = chrono::steady_clock::now();
    stdDevGPU(d_array, N, meanGPU, ans_GPU);
    end = chrono::steady_clock::now();

    cout << "Standard deviation value: " << *ans_GPU << '\n';
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    return 0;
}

CPU STATISTICS:
Sum value: 261845517
Average value: 499
Time taken: 1412 microseconds
Standard deviation value: 34
Time taken: 1401 microseconds

GPU STATISTICS:
Sum value: 261845517
Average value: 499
Time taken: 151 microseconds
Standard deviation value: 34
Time taken: 220 microseconds



## **VECTOR AND MATRIX OPERATIONS**

### **VECTOR ADDITION**

In [ ]:
%%cu
#include <iostream>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <time.h>
#include <chrono>
using namespace std;


__global__ void addGPU(int *x, int *y, int *z) {
    int idx = blockIdx.x;
    z[idx] = x[idx] + y[idx];
}

void addCPU(int *x, int *y, int *z, int n) {
    for(int i = 0; i < n; i++) z[i] = x[i] + y[i];
}

int main() {
    srand(time(0));
    int N = (1<<20);
    int *h_array_1, *h_array_2, *h_array_3;
    int *d_array_1, *d_array_2, *d_array_3;
    
    h_array_1 = new int[N];
    h_array_2 = new int[N];
    h_array_3 = new int[N];
    cudaMalloc((void**)&d_array_1, N * sizeof(int));
    cudaMalloc((void**)&d_array_2, N * sizeof(int));
    cudaMalloc((void**)&d_array_3, N * sizeof(int));

    for(int i = 0; i < N; i++) {
        h_array_1[i] = rand();
        h_array_2[i] = rand();
        h_array_3[i] = 0;
    }
    cudaMemcpy(d_array_1, h_array_1, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_array_2, h_array_2, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_array_3, h_array_3, N * sizeof(int), cudaMemcpyHostToDevice);

    auto start = chrono::steady_clock::now();
    addCPU(h_array_1, h_array_2, h_array_3, N);
    auto end = chrono::steady_clock::now();

    cout << "VECTOR ADDITION:\n\n";

    cout << "CPU STATISTICS:\n";
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    start = chrono::steady_clock::now();
    addGPU<<<N, 1>>>(d_array_1, d_array_2, d_array_3);
    end = chrono::steady_clock::now();

    cout << "GPU STATISTICS:\n";
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    return 0;
}

VECTOR ADDITION:

CPU STATISTICS:
Time taken: 3660 microseconds
GPU STATISTICS:
Time taken: 197 microseconds



### **MATRIX ADDITION**

In [ ]:
%%cu
#include <iostream>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <time.h>
#include <chrono>
#define N 512
#define M 512
using namespace std;


__global__ void addGPU(int *x, int *y, int *z) {
    int idx = blockIdx.x * M + blockIdx.y;
    z[idx] = x[idx] + y[idx];
}

void addCPU(int x[N][M], int y[N][M], int z[N][M]) {
    for(int i = 0; i < N; i++) for(int j = 0; j < M; j++) z[i][j] = x[i][j] + y[i][j];
}

int main() {
    srand(time(0));
    int h_array_1[N][M], h_array_2[N][M], h_array_3[N][M];
    int *d_array_1, *d_array_2, *d_array_3;

    cudaMalloc((void**)&d_array_1, N * M * sizeof(int));
    cudaMalloc((void**)&d_array_2, N * M * sizeof(int));
    cudaMalloc((void**)&d_array_3, N * M * sizeof(int));

    for(int i = 0; i < N; i++) for(int j = 0; j < M; j++) {
        h_array_1[i][j] = rand() % 1000;
        h_array_2[i][j] = rand() % 1000;
        h_array_3[i][j] = 0;
    }
    cudaMemcpy(d_array_1, h_array_1, N * M * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_array_2, h_array_2, N * M * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_array_3, h_array_3, N * M * sizeof(int), cudaMemcpyHostToDevice);

    auto start = chrono::steady_clock::now();
    addCPU(h_array_1, h_array_2, h_array_3);
    auto end = chrono::steady_clock::now();

    cout << "MATRIX ADDITION:\n\n";

    cout << "CPU STATISTICS:\n";
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";
    
    dim3 grid(N,M);
    start = chrono::steady_clock::now();
    addGPU<<<grid, 1>>>(d_array_1, d_array_2, d_array_3);
    end = chrono::steady_clock::now();

    cout << "GPU STATISTICS:\n";
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    return 0;
}

MATRIX ADDITION:

CPU STATISTICS:
Time taken: 905 microseconds
GPU STATISTICS:
Time taken: 145 microseconds



### **MATRIX MULTIPLICATION**

In [ ]:
%%cu
#include <iostream>
#include <cstdio>
#include <cstdlib>
#include <cuda_runtime.h>
#include <time.h>
#include <chrono>
#define N1 512
#define M1 512
#define N2 512
#define M2 512
using namespace std;


__global__ void multiplyGPU(int *x, int *y, int *z) {
    int i = blockIdx.x;
    int j = blockIdx.y;
    for(int k = 0; k < M1; k++) z[M2*i + j] += x[M1*i + k] * y[M2*k + j];
}

void multiplyCPU(int x[N1][M1], int y[N2][M2], int z[N1][M2]) {
    for(int i = 0; i < N1; i++) for(int j = 0; j < M2; j++)
      for(int k = 0; k < M1; k++) z[i][j] += x[i][k] * y[k][j];
}

int main() {
    srand(time(0));
    int h_array_1[N1][M1], h_array_2[N2][M2], h_array_3[N1][M2];
    int *d_array_1, *d_array_2, *d_array_3;

    cudaMalloc((void**)&d_array_1, N1 * M1 * sizeof(int));
    cudaMalloc((void**)&d_array_2, N2 * M2 * sizeof(int));
    cudaMalloc((void**)&d_array_3, N1 * M2 * sizeof(int));

    for(int i = 0; i < N1; i++) for(int j = 0; j < M1; j++) h_array_1[i][j] = rand() % 1000;
    for(int i = 0; i < N2; i++) for(int j = 0; j < M2; j++) h_array_2[i][j] = rand() % 1000;
    for(int i = 0; i < N1; i++) for(int j = 0; j < M2; j++) h_array_3[i][j] = 0;

    cudaMemcpy(d_array_1, h_array_1, N1 * M1 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_array_2, h_array_2, N2 * M2 * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_array_3, h_array_3, N1 * M2 * sizeof(int), cudaMemcpyHostToDevice);

    auto start = chrono::steady_clock::now();
    multiplyCPU(h_array_1, h_array_2, h_array_3);
    auto end = chrono::steady_clock::now();

    cout << "MATRIX MULTIPLICATION:\n\n";

    cout << "CPU STATISTICS:\n";
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";
    
    dim3 grid(N1,M2);
    start = chrono::steady_clock::now();
    multiplyGPU<<<grid, 1>>>(d_array_1, d_array_2, d_array_3);
    end = chrono::steady_clock::now();

    cout << "GPU STATISTICS:\n";
    cout << "Time taken: " << chrono::duration_cast<chrono::microseconds>(end - start).count() << " microseconds\n";

    return 0;
}

MATRIX MULTIPLICATION:

CPU STATISTICS:
Time taken: 1205479 microseconds
GPU STATISTICS:
Time taken: 202 microseconds

